## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [1]:
import torch
import numpy as np
import torch.nn as nn

As before, we represent the inputs and targets and matrices. We will use slightly larget dataset to also clarify how to deal with large
dataset in batches.

In [4]:
# Input (temp, rainfall, humidity)
inputs = torch.tensor([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype=torch.float32)

# Targets (apples, oranges)
targets = torch.tensor([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype=torch.float32)

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [5]:
from torch.utils.data import TensorDataset

In [6]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [7]:
from torch.utils.data import DataLoader

In [20]:
# Define data loader
# It is not effeicient to load all data in one go
# rather we load data in batches. batch_size 5 means
# we would have 5 samples in each batch of data
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
# this should create 3 batches each having 5 data

We can use the data loader in a `for` loop. Let's look at an example.

In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

In [60]:
for xb,yb in train_dl:
    print(f"{xb}\n{yb}")
    print()
# run this multiple times to see the working of DataLoader

tensor([[ 73.,  67.,  43.],
        [ 68.,  97.,  70.],
        [103.,  43.,  36.],
        [102.,  43.,  37.],
        [ 91.,  87.,  65.]])
tensor([[ 56.,  70.],
        [102., 120.],
        [ 20.,  38.],
        [ 22.,  37.],
        [ 80., 102.]])

tensor([[101.,  44.,  37.],
        [ 91.,  88.,  64.],
        [ 88., 134.,  59.],
        [ 73.,  66.,  44.],
        [ 74.,  66.,  43.]])
tensor([[ 21.,  38.],
        [ 81., 101.],
        [118., 132.],
        [ 57.,  69.],
        [ 57.,  69.]])

tensor([[ 69.,  96.,  70.],
        [ 68.,  96.,  71.],
        [ 87., 134.,  58.],
        [ 87., 135.,  57.],
        [ 92.,  87.,  64.]])
tensor([[103., 119.],
        [104., 118.],
        [119., 133.],
        [118., 134.],
        [ 82., 100.]])



## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [21]:
# Define model
model = nn.Linear(3, 2)  # inputs, targets
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4592,  0.3581, -0.0990],
        [ 0.4693,  0.1024,  0.2262]], requires_grad=True)
Parameter containing:
tensor([-0.5112,  0.0447], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [22]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.4592,  0.3581, -0.0990],
         [ 0.4693,  0.1024,  0.2262]], requires_grad=True),
 Parameter containing:
 tensor([-0.5112,  0.0447], requires_grad=True)]

We can use the model to generate predictions in the same way as before.

In [23]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-14.2956,  50.8916],
        [-17.1192,  66.2396],
        [  1.7860,  67.7152],
        [-35.6149,  60.6876],
        [ -4.7448,  58.0907],
        [-15.1130,  51.2586],
        [-17.5763,  66.3634],
        [  1.2278,  68.4107],
        [-34.7975,  60.3206],
        [ -4.3845,  57.8475],
        [-14.7527,  51.0154],
        [-17.9366,  66.6065],
        [  2.2431,  67.5914],
        [-35.9752,  60.9307],
        [ -3.9274,  57.7238]], grad_fn=<AddmmBackward0>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [24]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities. 

In [25]:
# Define loss function
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [26]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(5314.9214, grad_fn=<MseLossBackward0>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

In [28]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [32]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            # print(xb,yb)
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}')

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [33]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 23.3678
Epoch [20/100], Loss: 21.4010
Epoch [30/100], Loss: 20.7087
Epoch [40/100], Loss: 15.8776
Epoch [50/100], Loss: 11.4028
Epoch [60/100], Loss: 11.3292
Epoch [70/100], Loss: 17.6538
Epoch [80/100], Loss: 11.7817
Epoch [90/100], Loss: 8.1945
Epoch [100/100], Loss: 13.1941


Let's generate predictions using our model and verify that they're close to our targets.

In [48]:
# Generate predictions and compare with targets
preds = model(inputs)
preds,targets

(tensor([[ 57.1409,  70.7898],
         [ 80.0950,  98.7419],
         [122.2651, 136.2448],
         [ 22.2292,  39.8011],
         [ 97.5685, 114.1894],
         [ 55.8639,  69.7003],
         [ 79.5843,  98.4942],
         [122.3663, 136.6665],
         [ 23.5062,  40.8906],
         [ 98.3348, 115.0312],
         [ 56.6302,  70.5421],
         [ 78.8181,  97.6524],
         [122.7758, 136.4925],
         [ 21.4630,  38.9593],
         [ 98.8455, 115.2789]], grad_fn=<AddmmBackward0>),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.],
         [ 57.,  69.],
         [ 80., 102.],
         [118., 132.],
         [ 21.,  38.],
         [104., 118.],
         [ 57.,  69.],
         [ 82., 100.],
         [118., 134.],
         [ 20.,  38.],
         [102., 120.]]))

## Machine Learning vs. Classical Programming

The approach we've taken in this tutorial is very different from programming as you might know it. Usually, we write programs that take some inputs, perform some operations, and return a result. 

However, in this notebook, we've defined a "model" that assumes a specific relationship between the inputs and the outputs, expressed using some unknown parameters (weights & biases). We then show the model some know inputs and outputs and _train_ the model to come up with good values for the unknown parameters. Once trained, the model can be used to compute the outputs for new inputs.

This paradigm of programming is known as _machine learning_, where we use data to figure out the relationship between inputs and outputs. _Deep learning_ is a branch of machine learning that uses matrix operations, non-linear activation functions and gradient descent to build and train models. Andrej Karpathy, the director of AI at Tesla Motors, has written a great blog post on this topics, titled [Software 2.0](https://medium.com/@karpathy/software-2-0-a64152b37c35).

This picture from book [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python) by Francois Chollet captures the difference between classical programming and machine learning:

![](https://i.imgur.com/oJEQe7k.png)

Keep this picture in mind as you work through the next few tutorials. 



## Exercises and Further Reading

We've covered the following topics in this tutorial:

- Introduction to linear regression and gradient descent
- Implementing a linear regression model using PyTorch tensors
- Training a linear regression model using the gradient descent algorithm
- Implementing gradient descent and linear regression using PyTorch built-in


Here are some resources for learning more about linear regression and gradient descent:

* An visual & animated explanation of gradient descent: https://www.youtube.com/watch?v=IHZwWFHWa-w

* For a more detailed explanation of derivates and gradient descent, see [these notes from a Udacity course](https://storage.googleapis.com/supplemental_media/udacityu/315142919/Gradient%20Descent.pdf). 

* For an animated visualization of how linear regression works, [see this post](https://hackernoon.com/visualizing-linear-regression-with-pytorch-9261f49edb09).

* For a more mathematical treatment of matrix calculus, linear regression and gradient descent, you should check out [Andrew Ng's excellent course notes](https://github.com/Cleo-Stanford-CS/CS229_Notes/blob/master/lectures/cs229-notes1.pdf) from CS229 at Stanford University.

* To practice and test your skills, you can participate in the [Boston Housing Price Prediction](https://www.kaggle.com/c/boston-housing) competition on Kaggle, a website that hosts data science competitions.

With this, we complete our discussion of linear regression in PyTorch, and we’re ready to move on to the next topic: [Working with Images & Logistic Regression](https://jovian.ai/aakashns/03-logistic-regression).